In [2]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role
from dotenv import load_dotenv
load_dotenv()  # 自动加载 .env 文件中的环境变量
dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")

# 本地函数：模拟天气查询的函数
def get_current_weather(location, unit="摄氏度"):
    # 获取指定地点的天气
    temperature = -1
    if '杭州' in location or 'Hangzhou' in location:
        temperature = 10
    if location=='上海':
        temperature = 36
    if location=='深圳':
        temperature = 37
    weather_info = {
        "location": location,
        "temperature": temperature,
        "unit": unit,
        "forecast": ["晴天", "微风"],
    }
    return json.dumps(weather_info)

# 定义 Function Call 的描述
functions = [
    {
      'name': 'get_current_weather',
      'description': 'Get the current weather in a given location.', # 定义function的描述
      'parameters': { # 定义function的参数
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The city and state, e.g. San Francisco, CA'
                },
                'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}
            },
        'required': ['location']
      }
    }
]

# 封装模型响应函数
def get_response(messages):
    try:
        response = dashscope.Generation.call(
            model='qwen-turbo',
            messages=messages,
            functions=functions,
            result_format='message'
        )
        return response
    except Exception as e:
        print(f"API调用出错: {str(e)}")
        return None

# 主流程函数：使用function call进行QA
def run_conversation():
     # Step 1, 发送用户查询
    query = "杭州的天气怎样"
    messages=[
        {"role": "user", "content": query}
    ]
    response = get_response(messages) # 调用模型，得到第一次响应
    if not response or not response.output:
        print("获取响应失败")
        return None
        
    print('response=', response)
    
    message = response.output.choices[0].message
    messages.append(message)
    print('message=', message)
    
    # Step 2, 判断用户是否要call function【主流大模型都支持 function_call 类似机制】
    if 'function_call' in message and message.function_call:
        function_call = message.function_call
        tool_name = function_call['name']

        # Step 3, 执行function call
        # 解析大模型返回的 function_call 字段，提取参数
        arguments = json.loads(function_call['arguments'])
        print('arguments=', arguments)
        #调用本地的 get_current_weather 函数，得到天气信息
        tool_response = get_current_weather(
            location=arguments.get('location'),
            unit=arguments.get('unit'),
        )
        # 把本地函数的结果作为一条新的消息，追加到 messages 列表
        tool_info = {"role": "function", "name": tool_name, "content": tool_response}
        print('tool_info=', tool_info)
        messages.append(tool_info)
        print('messages=', messages)
        
        #Step 4, 得到第二次响应
        response = get_response(messages)
        if not response or not response.output:
            print("获取第二次响应失败")
            return None
            
        print('response=', response)
        message = response.output.choices[0].message
        return message
    return message

if __name__ == "__main__":
    result = run_conversation()
    if result:
        print("最终结果:", result)
    else:
        print("对话执行失败")

response= {"status_code": 200, "request_id": "0a88c21f-9388-9c30-90f1-65db96ce01ca", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "{\"name\": \"get_current_weather\", \"arguments\": {\"location\": \"杭州\"}}"}}]}, "usage": {"input_tokens": 194, "output_tokens": 17, "total_tokens": 211, "prompt_tokens_details": {"cached_tokens": 0}}}
message= {"role": "assistant", "content": "{\"name\": \"get_current_weather\", \"arguments\": {\"location\": \"杭州\"}}"}
最终结果: {"role": "assistant", "content": "{\"name\": \"get_current_weather\", \"arguments\": {\"location\": \"杭州\"}}"}
